In [17]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import urllib.request

import numpy as np
import tensorflow as tf

In [18]:
# Data sets
IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

In [21]:
# If the training and test sets aren't stored locally, download them.
if not os.path.exists(IRIS_TRAINING):
    raw = str(urllib.request.urlopen(IRIS_TRAINING_URL).read(),'utf-8') # python 3
    with open(IRIS_TRAINING, "w") as f:
        f.write(raw)

if not os.path.exists(IRIS_TEST):
    raw = str(urllib.request.urlopen(IRIS_TEST_URL).read(),'utf-8')
    with open(IRIS_TEST, "w") as f:
        f.write(raw)

In [22]:
# Load datasets.
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
      filename=IRIS_TRAINING,
      target_dtype=np.int,
      features_dtype=np.float32)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
      filename=IRIS_TEST,
      target_dtype=np.int,
      features_dtype=np.float32)

In [23]:
# Specify that all features have real-value data
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)]

In [24]:
# Build 3 layer DNN with 10, 20, 10 units respectively.
classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,
                                              hidden_units=[10, 20, 10],
                                              n_classes=3,
                                              model_dir="/tmp/iris_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_is_chief': True, '_environment': 'local', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_master': '', '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000021AEA2EA7F0>, '_save_checkpoints_secs': 600, '_evaluation_master': '', '_save_checkpoints_steps': None, '_task_type': None, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_tf_random_seed': None}


In [25]:
# Define the training inputs
def get_train_inputs():
    x = tf.constant(training_set.data)
    y = tf.constant(training_set.target)

    return x, y

In [26]:
# Fit model.
classifier.fit(input_fn=get_train_inputs, steps=2000)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/iris_model\model.ckpt.
INFO:tensorflow:loss = 1.09723, step = 1
INFO:tensorflow:global_step/sec: 1279.94
INFO:tensorflow:loss = 0.174579, step = 101
INFO:tensorflow:global_step/sec: 1279.94
INFO:tensorflow:loss = 0.087889, step = 201
INFO:tensorflow:global_step/sec: 1279.93
INFO:tensorflow:loss = 0.0692917, step = 301
INFO:tensorflow:global_step/sec: 1279.91
INFO:tensorflow:loss = 0.0623212, step = 401
INFO:tensorflow:global_step/sec: 1279.94
INFO:tensorflow:loss = 0.0579588, step = 501
INFO:tensorflow:global_step/sec: 1279.93
INFO:tensorflow:loss = 0.0551561, step = 60

DNNClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x0000021AEA2EA780>, 'input_layer_min_slice_size': None, 'feature_columns': (_RealValuedColumn(column_name='', dimension=4, default_value=None, dtype=tf.float32, normalizer=None),), 'gradient_clip_norm': None, 'hidden_units': [10, 20, 10], 'activation_fn': <function relu at 0x0000021AE8BFC268>, 'dropout': None, 'optimizer': None, 'embedding_lr_multipliers': None})

In [27]:
# Define the test inputs
def get_test_inputs():
    x = tf.constant(test_set.data)
    y = tf.constant(test_set.target)

    return x, y

In [28]:
# Evaluate accuracy.
accuracy_score = classifier.evaluate(input_fn=get_test_inputs,
                                       steps=1)["accuracy"]
print("\nTest Accuracy: {0:f}\n".format(accuracy_score))

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-04-30-21:55:59
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2017-04-30-21:55:59
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.966667, auc = 0.998333, global_step = 2000, loss = 0.0587128

Test Accuracy: 0.966667



In [29]:
# Classify two new flower samples.
def new_samples():
    return np.array(
      [[6.4, 3.2, 4.5, 1.5],
       [5.8, 3.1, 5.0, 1.7]], dtype=np.float32)

In [30]:
predictions = list(classifier.predict(input_fn=new_samples))
print(
      "New Samples, Class Predictions:    {}\n"
      .format(predictions))

New Samples, Class Predictions:    [1, 2]

